In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os
import pandas as pd
import os
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
import cv2

In [2]:
torch.set_float32_matmul_precision('high')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [4]:
target_files = ["1652875851.3497071", "1652875901.3107166", "1652876013.741493", "1652876206.2541456", "1652876485.8123376", "1652959186.4507334",
                "1652959347.972946", "1653042695.4914637", "1653042775.5213027", "1653043202.5073502", "1653043345.3415065", "1653043428.8546412", "1653043549.5187616"]

In [5]:
random_transforms = transforms.Compose([
    transforms.ColorJitter(brightness=.5),
    transforms.RandomResizedCrop(IMG_WIDTH,(0.9,0.9),(1,1)),
    transforms.RandomRotation(10)
])

In [6]:
def preprocess(img: np.ndarray) -> np.ndarray:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape
    roi_height = height // 2
    roi = gray[height - roi_height:height, :]
    mean_val = np.mean(roi)
    std_val = np.std(roi)
    lower_threshold = max(0, mean_val - std_val)
    upper_threshold = min(255, mean_val + std_val)
    edges = cv2.Canny(roi, lower_threshold, upper_threshold)
    result = np.zeros((height, width), dtype=np.uint8)
    result[height - roi_height:height, :] = edges
    kernel = np.ones((2, 2), np.uint8)
    result = cv2.dilate(result, kernel, iterations=1)
    result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR).astype(np.float32) / 255.0
    return result

In [7]:
class ImageDataset(Dataset):
    def __init__(self, files, preprocess = False):
        self.image_dirs = files
        self.targets = pd.DataFrame(columns=['path', 'forward', 'left'])
        self.preprocess = preprocess
        for image_dir in self.image_dirs:
            def change_name(number:float) -> str:
                true_num = int(number)
                return f"{os.path.join(image_dir, f'{true_num:04}.jpg')}"
            frame = pd.read_csv(os.path.join('dataset', f'{image_dir}.csv'), names=['path', 'forward', 'left'])
            frame['path'] = frame['path'].apply(change_name)
            self.targets = pd.concat([self.targets, frame])
            self.paths:pd.Series[str] = self.targets.pop('path')
            break

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        image_path = self.paths[idx]
        target = torch.tensor(self.targets.iloc[idx].values,dtype=torch.float32)
        image = Image.open(os.path.join('dataset', image_path))
        image = np.asarray(image)
        if self.preprocess:
            image = preprocess(image)
        image = transforms.ToTensor()(image)
        image = image.to(device)
        target = target.to(device)
        return image, target


In [8]:
class DrivingModel(nn.Sequential):
    def __init__(self):
        super(DrivingModel, self).__init__()
        self.input_shape = (1, 3, IMG_WIDTH, IMG_HEIGHT)
        
        self.append(nn.Conv2d(3, 16, kernel_size=3, padding=1))
        self.append(nn.ReLU())

        self.append(nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.append(nn.Conv2d(16, 32, kernel_size=3, padding=1))
        self.append(nn.ReLU())
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.append(nn.Conv2d(32, 64, kernel_size=3, padding=1))
        self.append(nn.ReLU())
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.append(nn.Conv2d(64, 128, kernel_size=3, padding=1))
        self.append(nn.ReLU())
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))

        self.append(nn.Conv2d(128, 128, kernel_size=3, padding=1))
        self.append(nn.ReLU())
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))

        self.append(nn.Flatten())

        self.append(nn.Linear(128 * 7 * 7, 32))
        self.append(nn.ReLU())
        self.append(nn.Linear(32, 2))

    def forward(self, x):
        x = x.view(-1, 3, 224, 224)
        for layer in self:
            x = layer(x)
        return x

In [9]:
def train(model:nn.Module, train_loader, val_loader, criterion, optimizer) -> nn.Module:
    num_epochs = 1000
    best_val_loss = None
    counter = 20
    val_loss = None
    train_loss = None
    p_bar = tqdm(range(num_epochs), desc="Epochs")
    for epoch in p_bar:
        p_bar.set_postfix({'val_loss': val_loss, 'train_loss': train_loss})
        train_loss = 0
        model.train()
        for images, targets in tqdm(train_loader, desc="Training", leave=False):
            optimizer.zero_grad()
            images = random_transforms(images)
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        val_loss = 0
        model.eval()
        for images, targets in tqdm(val_loader, desc="Validation", leave=False):
            images = images.to(device)
            targets = targets.to(device)
            with torch.inference_mode():
                outputs = model(images)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        if best_val_loss is None:
            best_val_loss = val_loss
        elif best_val_loss < val_loss:
            counter -= 1
        else:
            best_val_loss = val_loss
            counter = 20
        if not counter:
            break
    return model

In [10]:
import onnx

def pipeline(preprocess=False):
    img_dataset = ImageDataset(target_files, preprocess=preprocess)
    train_dataset, val_dataset = random_split(img_dataset, [0.7, 0.3])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    model = DrivingModel().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())
    model = train(model, train_loader, val_loader, criterion, optimizer)
    torch.save(model.state_dict(), f'model_{int(preprocess)}.pth')

    x = torch.randn(1, 3, IMG_WIDTH, IMG_HEIGHT).to(device)
    torch.onnx.export(model,
                      (x,),
                      f'model_{int(preprocess)}.onnx',
                      export_params=True,
                      opset_version=11,
                      do_constant_folding=True,
                      input_names=['input'],
                      output_names=['output'],
                      dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
                      )
    onnx_model = onnx.load(f'model_{int(preprocess)}.onnx')
    onnx.checker.check_model(onnx_model)
    onnx.save_model(onnx_model, f'model_{int(preprocess)}.onnx')

In [11]:
pipeline()
pipeline(preprocess=True)

/tmp/ipykernel_9377/1345956773.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.targets = pd.concat([self.targets, frame])


Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Training:   0%|          | 0/4 [00:00<?, ?it/s]

/home/anatol/Documents/GitHub/RoboticsII-Jetbot/.venv/lib/python3.13/site-packages/torchvision/transforms/functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


Validation:   0%|          | 0/2 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/2 [00:00<?, ?it/s]

Training:   0%|          | 0/4 [00:00<?, ?it/s]

Validation:   0%|          | 0/2 [00:00<?, ?it/s]

Training:   0%|          | 0/4 [00:00<?, ?it/s]

Validation:   0%|          | 0/2 [00:00<?, ?it/s]